# Battle of the Neighbourhoods Notebook

In [134]:
import pandas as pd
import numpy as np
import requests

In [90]:
price = pd.read_excel('https://data.sa.gov.au/data/dataset/0d447195-1158-4a3c-8cc7-0e333b87eb72/resource/82ed30d2-1dae-4469-ace2-5ddaab7a8d63/download/lsg_stats_2019_q4.xlsx')
price.head()

,City,Suburb,Sales 4Q 2018,Median 4Q 2018,Sales 4Q 2019,Median 4Q 2019,Median Change
0,ADELAIDE,ADELAIDE,5.0,1590000.0,4.0,770117.0,-0.515650
1,ADELAIDE,NORTH ADELAIDE,9.0,1395000.0,10.0,1352500.0,-0.030466
2,ADELAIDE HILLS,ALDGATE,9.0,747500.0,14.0,753250.0,0.007692
3,ADELAIDE HILLS,ASHTON,NaN,NaN,NaN,NaN,NaN
4,ADELAIDE HILLS,BASKET RANGE,1.0,640000.0,1.0,375000.0,-0.414062


In [110]:
price_clean = price.drop(price.columns[[2,3,6]], axis=1)
price_clean.dropna(inplace=True)
price_clean.sort_values(by=['Suburb'], inplace=True)
price_clean.reset_index(inplace=True)
price_clean.drop(price_clean.columns[[0]], axis=1, inplace=True)
price_clean.head()

,City,Suburb,Sales 4Q 2019,Median 4Q 2019
0,ONKAPARINGA,ABERFOYLE PARK,34.0,443000.0
1,ADELAIDE,ADELAIDE,4.0,770117.0
2,CHARLES STURT,ALBERT PARK,10.0,505000.0
3,PORT ADELAIDE ENFIELD,ALBERTON,11.0,410000.0
4,ADELAIDE HILLS,ALDGATE,14.0,753250.0


In [113]:
geo = pd.read_csv('https://raw.githubusercontent.com/SilentNumber/Coursera_Capstone/master/geo.csv')
geo.reset_index()
geo['Suburb'] = geo['Suburb'].str.upper()
geo_clean = geo.drop(geo.columns[[0,2]], axis=1)
geo_clean.sort_values(by=['Suburb'], inplace=True)
geo_clean.reset_index(inplace=True)
geo_clean.drop(geo_clean.columns[[0]], axis=1, inplace=True)
geo_clean.head()

,Suburb,Lat,Long
0,ABERFOYLE PARK,-35.08,138.59
1,ADELAIDE,-34.93,138.60
2,ADELAIDE,-35.12,139.27
3,ADELAIDE,-35.12,139.27
4,ADELAIDE,-35.12,139.27


In [165]:
merged = pd.merge(price_clean, geo_clean[['Suburb', 'Lat', 'Long']], on='Suburb')
merged = merged.drop_duplicates(subset='Suburb', keep="first")
merged = merged[merged.Lat != 0]
merged.reset_index(inplace=True)
merged.drop(merged.columns[[0]], axis=1, inplace=True)
merged.rename(columns={"Suburb": "Neighbourhood"}, inplace=True)
merged.head()

,City,Neighbourhood,Sales 4Q 2019,Median 4Q 2019,Lat,Long
0,ONKAPARINGA,ABERFOYLE PARK,34.0,443000.0,-35.08,138.59
1,ADELAIDE,ADELAIDE,4.0,770117.0,-34.93,138.60
2,CHARLES STURT,ALBERT PARK,10.0,505000.0,-34.88,138.52
3,PORT ADELAIDE ENFIELD,ALBERTON,11.0,410000.0,-34.88,138.52
4,ADELAIDE HILLS,ALDGATE,14.0,753250.0,-35.01,138.73


In [131]:
!pip install folium

     |████████████████████████████████| 92kB 7.3MB/s eta 0:00:011


In [132]:
CLIENT_ID = 'LS51W0YRQKTSEWE0HKG4JH3W10EAFVKFUTSODOW2YXCLAH3Y'
CLIENT_SECRET = 'S15CEJEAW5OF5EMNEGMLLM2RMVJVFRDEAGQ5XJ4YSNQHBU31'
VERSION = '20180605'
LIMIT = 100
radius = 500

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [148]:
adelaide_venues = getNearbyVenues(names=merged['Suburb'],
                                   latitudes=merged['Lat'],
                                   longitudes=merged['Long']
                                  )

ABERFOYLE PARK
ADELAIDE
ALBERT PARK
ALBERTON
ALDGATE
ALDINGA
ALDINGA BEACH
ALLENBY GARDENS
ANDREWS FARM
ANGLE PARK
ANGLE VALE
ASCOT PARK
ATHELSTONE
ATHOL PARK
AULDANA
BANKSIA PARK
BASKET RANGE
BEAUMONT
BEDFORD PARK
BELAIR
BELLEVUE HEIGHTS
BEULAH PARK
BEVERLEY
BIRKENHEAD
BLACK FOREST
BLACKWOOD
BLAIR ATHOL
BLAKEVIEW
BOWDEN
BRAHMA LODGE
BRIDGEWATER
BRIGHTON
BROADVIEW
BROMPTON
BROOKLYN PARK
BURNSIDE
BURTON
CAMDEN PARK
CAMPBELLTOWN
CHANDLERS HILL
CHELTENHAM
CHERRYVILLE
CHRISTIE DOWNS
CHRISTIES BEACH
CLAPHAM
CLARENCE GARDENS
CLARENCE PARK
CLEARVIEW
CLOVELLY PARK
COLLEGE PARK
COLLINSWOOD
COLONEL LIGHT GARDENS
COROMANDEL VALLEY
COWANDILLA
CRAFERS
CRAFERS WEST
CRAIGBURN FARM
CRAIGMORE
CROYDON
CROYDON PARK
CUMBERLAND PARK
DARLINGTON
DAVOREN PARK
DAW PARK
DERNANCOURT
DEVON PARK
DIREK
DOVER GARDENS
DUDLEY PARK
DULWICH
EASTWOOD
EDEN HILLS
EDWARDSTOWN
ELIZABETH
ELIZABETH DOWNS
ELIZABETH EAST
ELIZABETH GROVE
ELIZABETH NORTH
ELIZABETH PARK
ELIZABETH SOUTH
ELIZABETH VALE
ENFIELD
ERINDALE
ETHELTON
EVAND

In [149]:
adelaide_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,ABERFOYLE PARK,-35.08,138.59,Subway,-35.076461,138.591040,Sandwich Place
1,ABERFOYLE PARK,-35.08,138.59,Domino's Pizza,-35.076607,138.591152,Pizza Place
2,ABERFOYLE PARK,-35.08,138.59,Woolworths,-35.075922,138.591942,Supermarket
3,ABERFOYLE PARK,-35.08,138.59,Lyttle Orchid Clones,-35.080092,138.593946,Garden
4,ADELAIDE,-34.93,138.60,Adelaide Central Market,-34.929479,138.597674,Farmers Market


In [150]:
adelaide_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
ABERFOYLE PARK,4,4,4,4,4,4
ADELAIDE,100,100,100,100,100,100
ALBERT PARK,3,3,3,3,3,3
ALBERTON,3,3,3,3,3,3
ALDGATE,4,4,4,4,4,4
ALDINGA,5,5,5,5,5,5
ALDINGA BEACH,5,5,5,5,5,5
ALLENBY GARDENS,14,14,14,14,14,14
ANDREWS FARM,3,3,3,3,3,3


In [151]:
print('There are {} uniques categories.'.format(len(adelaide_venues['Venue Category'].unique())))

There are 165 uniques categories.


In [152]:
# one hot encoding
adelaide_onehot = pd.get_dummies(adelaide_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
adelaide_onehot['Neighbourhood'] = adelaide_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [adelaide_onehot.columns[-1]] + list(adelaide_onehot.columns[:-1])
adelaide_onehot = adelaide_onehot[fixed_columns]

adelaide_onehot.head()

,Neighbourhood,Afghan Restaurant,American Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Garage,...,Tram Station,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Winery,Wings Joint,Xinjiang Restaurant,Zoo
0,ABERFOYLE PARK,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ABERFOYLE PARK,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ABERFOYLE PARK,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ABERFOYLE PARK,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ADELAIDE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [153]:
adelaide_grouped = adelaide_onehot.groupby('Neighbourhood').mean().reset_index()
adelaide_grouped

,Neighbourhood,Afghan Restaurant,American Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Garage,...,Tram Station,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Winery,Wings Joint,Xinjiang Restaurant,Zoo
0,ABERFOYLE PARK,0.0,0.0,0.00,0.0,0.00,0.000000,0.000000,0.000000,0.0,...,0.00,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.00,0.0
1,ADELAIDE,0.0,0.0,0.01,0.0,0.00,0.060000,0.000000,0.010000,0.0,...,0.01,0.0,0.000000,0.030000,0.04,0.000000,0.000000,0.0,0.01,0.0
2,ALBERT PARK,0.0,0.0,0.00,0.0,0.00,0.000000,0.000000,0.000000,0.0,...,0.00,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.00,0.0
3,ALBERTON,0.0,0.0,0.00,0.0,0.00,0.000000,0.000000,0.000000,0.0,...,0.00,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.00,0.0
4,ALDGATE,0.0,0.0,0.00,0.0,0.00,0.000000,0.000000,0.000000,0.0,...,0.00,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.00,0.0
5,ALDINGA,0.0,0.0,0.00,0.0,0.00,0.000000,0.000000,0.000000,0.0,...,0.00,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.00,0.0
6,ALDINGA BEACH,0.0,0.0,0.00,0.0,0.00,0.000000,0.000000,0.000000,0.0,...,0.00,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.00,0.0
7,ALLENBY GARDENS,0.0,0.0,0.00,0.0,0.00,0.142857,0.000000,0.000000,0.0,...,0.00,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.00,0.0
8,ANDREWS FARM,0.0,0.0,0.00,0.0,0.00,0.000000,0.000000,0.000000,0.0,...,0.00,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.00,0.0
9,ANGLE PARK,0.0,0.0,0.00,0.0,0.00,0.000000,0.166667,0.000000,0.0,...,0.00,0.0,0.000000,0.500000,0.00,0.000000,0.000000,0.0,0.00,0.0


In [154]:
num_top_venues = 5

for hood in adelaide_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = adelaide_grouped[adelaide_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----ABERFOYLE PARK----
            venue  freq
0     Pizza Place  0.25
1     Supermarket  0.25
2          Garden  0.25
3  Sandwich Place  0.25
4     Music Venue  0.00


----ADELAIDE----
                venue  freq
0  Chinese Restaurant  0.09
1                Café  0.09
2         Coffee Shop  0.07
3    Asian Restaurant  0.06
4                 Pub  0.05


----ALBERT PARK----
               venue  freq
0       Home Service  0.33
1      Train Station  0.33
2           Pharmacy  0.33
3  Indian Restaurant  0.00
4       Neighborhood  0.00


----ALBERTON----
               venue  freq
0       Home Service  0.33
1      Train Station  0.33
2           Pharmacy  0.33
3  Indian Restaurant  0.00
4       Neighborhood  0.00


----ALDGATE----
               venue  freq
0  Indian Restaurant  0.25
1   Business Service  0.25
2         Restaurant  0.25
3      Jewelry Store  0.25
4  Afghan Restaurant  0.00


----ALDINGA----
                venue  freq
0              Bakery   0.4
1  Seafood Restaurant   0.2

In [155]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [191]:
num_top_venues = 3

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = adelaide_grouped['Neighbourhood']

for ind in np.arange(adelaide_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(adelaide_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,ABERFOYLE PARK,Sandwich Place,Pizza Place,Supermarket
1,ADELAIDE,Café,Chinese Restaurant,Coffee Shop
2,ALBERT PARK,Home Service,Train Station,Pharmacy
3,ALBERTON,Home Service,Train Station,Pharmacy
4,ALDGATE,Business Service,Indian Restaurant,Restaurant


In [215]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 10

adelaide_grouped_clustering = adelaide_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=1).fit(adelaide_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

,Afghan Restaurant,American Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Garage,Automotive Shop,...,Tram Station,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Winery,Wings Joint,Xinjiang Restaurant,Zoo
0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,...,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.0
1,0.0,0.0,0.01,0.0,0.0,0.06,0.0,0.01,0.0,0.0,...,0.01,0.0,0.0,0.03,0.04,0.0,0.0,0.0,0.01,0.0
2,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,...,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.0
3,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,...,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.0
4,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,...,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.0


In [192]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
neighborhoods_venues_sorted.head()

,Cluster Labels,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,8,ABERFOYLE PARK,Sandwich Place,Pizza Place,Supermarket
1,8,ADELAIDE,Café,Chinese Restaurant,Coffee Shop
2,1,ALBERT PARK,Home Service,Train Station,Pharmacy
3,1,ALBERTON,Home Service,Train Station,Pharmacy
4,8,ALDGATE,Business Service,Indian Restaurant,Restaurant


In [242]:
adelaide_merged = merged

# merge adelaide_grouped with adelaide_data to add latitude/longitude for each neighborhood
adelaide_merged = adelaide_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

adelaide_merged = adelaide_merged.dropna()
adelaide_merged = adelaide_merged.drop(adelaide_merged.columns[[0,2]], axis=1)

adelaide_median = adelaide_merged.drop(adelaide_merged.columns[[0,2,3,4,5,6,7]], axis=1)
adelaide_merged.head()

,Neighbourhood,Median 4Q 2019,Lat,Long,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,ABERFOYLE PARK,443000.0,-35.08,138.59,8.0,Sandwich Place,Pizza Place,Supermarket
1,ADELAIDE,770117.0,-34.93,138.60,8.0,Café,Chinese Restaurant,Coffee Shop
2,ALBERT PARK,505000.0,-34.88,138.52,1.0,Home Service,Train Station,Pharmacy
3,ALBERTON,410000.0,-34.88,138.52,1.0,Home Service,Train Station,Pharmacy
4,ALDGATE,753250.0,-35.01,138.73,8.0,Business Service,Indian Restaurant,Restaurant


In [200]:
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

adelaide_clusters = folium.Map(location=[-34.92866, 138.59863], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(adelaide_merged['Lat'], adelaide_merged['Long'], adelaide_merged['Neighbourhood'], adelaide_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(adelaide_clusters)
       
adelaide_clusters

In [247]:
adelaide_merged.loc[adelaide_merged['Cluster Labels'] == 0, adelaide_merged.columns[[0] + list(range(5, adelaide_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
14,AULDANA,Winery,Playground,Australian Restaurant
15,BANKSIA PARK,Playground,Bus Station,Zoo
176,MAGILL,Winery,Playground,Australian Restaurant
226,OSBORNE,Playground,Beach,Zoo
261,ROSSLYN PARK,Winery,Playground,Australian Restaurant
288,SKYE,Winery,Playground,Australian Restaurant
306,TAPEROO,Playground,Beach,Zoo
307,TEA TREE GULLY,Playground,Bus Station,Zoo
309,TERINGIE,Winery,Playground,Australian Restaurant
328,VISTA,Playground,Bus Station,Zoo


In [243]:
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=1).fit(adelaide_median)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

adelaide_median.insert(0, 'Cluster Labels', kmeans.labels_)
adelaide_median.head()

,Cluster Labels,Median 4Q 2019
0,3,443000.0
1,0,770117.0
2,1,505000.0
3,3,410000.0
4,0,753250.0


In [245]:
adelaide_median2 = pd.merge(adelaide_median, adelaide_merged[['Neighbourhood', 'Median\n4Q 2019', 'Lat', 'Long']])
adelaide_median2.drop_duplicates(inplace=True)
adelaide_median2.sort_values(by=['Neighbourhood'])
adelaide_median2.head()

,Cluster Labels,Median 4Q 2019,Neighbourhood,Lat,Long
0,3,443000.0,ABERFOYLE PARK,-35.08,138.59
1,0,770117.0,ADELAIDE,-34.93,138.60
2,1,505000.0,ALBERT PARK,-34.88,138.52
3,1,505000.0,BLAIR ATHOL,-34.86,138.60
6,3,410000.0,ALBERTON,-34.88,138.52


In [246]:
adelaide_median_clusters = folium.Map(location=[-34.92866, 138.59863], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(adelaide_median2['Lat'], adelaide_median2['Long'], adelaide_median2['Neighbourhood'], adelaide_median2['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(adelaide_median_clusters)
       
adelaide_median_clusters

In [250]:
adelaide_median.loc[adelaide_median['Cluster Labels'] == 0, adelaide_median.columns[list(range(0, adelaide_median.shape[1]))]]

,Cluster Labels,Median 4Q 2019
1,0,770117.0
4,0,753250.0
19,0,775505.5
21,0,740060.0
31,0,700000.0
37,0,710000.0
46,0,716250.0
56,0,707500.0
60,0,713000.0
63,0,700000.0
